# Week 9: IIR filters, FFT Windowing, Effects

### 12 March 2018

# Goals #

After doing this lab, you should be able to:
* Use an iir filter design tool to create a filter, then apply it to a sound
* Have hands-on experience using windowing for better FFT analysis and effects
* Understand how STFT can be used to implement effects in the frequency domain

# Part 1: IIR Filter design #

In this section, you will see how to use an IIR filter design tool, and compare it to an FIR filter.

## Loading some sound files ##
a. Start by grabbing the same music files you used last week. Put them in the same directory as this lab.

* http://www.doc.gold.ac.uk/~mas01rf/PMC2014-15/IPython/lab13/song1.wav
* http://www.doc.gold.ac.uk/~mas01rf/PMC2016-17/lab17/song2.wav

These are free sounds downloaded from

http://freemusicarchive.org/music/Jahzzar/Travellers_Guide/Siesta

and

http://freemusicarchive.org/music/Black_Ant/Free_Beats_Sel_3/Fater_Lee

b. Now load them into variables:

In [ ]:
song1 = wavReadMono("song1.wav")
song2 = wavReadMono("song2.wav")
#Listen to them if you'd like:
play(song1)
play(song2)

## Adding an alarm sound ## 
c. Use the code below (same as last week) to add an "alarm" sound to song1:

In [ ]:
#This code adds a 5000Hz sine wave to the first 1 second of song1, then stores it in a new variable:
t = np.arange(0, 1, 1/44100)
song1_5000 = song1[0:44100] + 0.3 * sin(2*pi*5000*t)
play(song1) #normal song
play(song1_5000) #song with alarm

Listen to the normal song, and the song with the alarm added, above. Make sure you can hear the alarm.

## Getting started with the filter design tool ## 
d. The `signal.iirfilter` function will create you a new IIR filter. At minimum, you need to tell it the order of the filter (i.e., how many feedforward and feedback coefficients), and the cutoff frequency (or frequencies, in the case of a bandpass/bandstop filter).

The cutoff frequency is expressed as a proportion of the Nyquist rate. For instance, a cutoff of 0.5 would be 1/2 the Nyquist rate, which would correspond to 11025Hz if you have a sample rate of 44100Hz (and therefore a Nyquist rate of 22050 Hz).

Use the `?` syntax below to read more about this filter design function.

In [ ]:
?signal.iirfilter

This function returns two coefficient arrays, b and a. Just like you saw in lecture, b contains the feedforward coefficients, and a contains the feedback coefficients. 

For example, the following code creates a highpass filter with order 10, with a cutoff at 0.3 times the Nyquist frequency. Then it prints out the coefficients.

In [ ]:
b, a = signal.iirfilter(10, 0.3, btype='highpass')
print "b is ", b
print "a is ", a

Note that you can copy these coefficients into your own code in any environment, and use the IIR filter equation to compute your filter!

But first you probably want to look at the frequency response to see what it looks like.

The `signal.freqz` function below computes the frequency response for frequencies from 0 to the Nyquist. It stores the frequency response in h and the frequencies in w:

In [ ]:
w, h = signal.freqz(b, a)

Let's define a function to plot the response, and we can re-use this function later:

In [ ]:
def plotResponse(w, h):
    fig = plt.figure()
    plt.title('Digital filter frequency response')
    ax1 = fig.add_subplot(111)

    plt.plot(w, 20 * np.log10(abs(h)), 'b') #plot in dB scale, wow!
    plt.ylabel('Amplitude [dB]', color='b')
    plt.xlabel('Frequency [rad/sample]')

    ax2 = ax1.twinx()
    angles = np.unwrap(np.angle(h))
    plt.plot(w, angles, 'g')
    plt.ylabel('Angle (radians)', color='g')
    plt.grid()
    plt.axis('tight')
    plt.show()

Finally, let's plot the response, using w and h:

In [ ]:
plotResponse(w,h)

In order to apply this filter to a sound, you can use the lfilter function:

In [2]:
filtered_sound = signal.lfilter(b, a, original_sound) #make sure you load a sound in original_sound first!

## Using the filter design tool ##

e. Now, use the space below to create a filter to filter out the alarm sound from `song1_5000`. Use visual inspection of the frequency response and your ears to figure out what a good filter will be.

In [ ]:
#Your work here









## Reflections ##

How does this IIR filter compare to the FIR filter you made in last week's lab? How many coefficients do you need to remove the alarm sound? Is it more or less than the FIR filter? How much delay is introduced?

In [ ]:
#Type your reflections here.

# Part 2: FFT and Windows #

Recall from lecture:

1. When you select a finite number of audio samples and take the FFT, you are implicitly applying a *rectangular window* to these samples. That is, you are multiplying your audio by a signal whose value is 0 for a while, then 1 for a while, then 0 for a while.
2. When you multiply two signals in the time domain, this is equivalent to convolving them in the frequency domain.
3. The spectrum of the rectangular window is ugly and bumpy (i.e., it has high side lobes). This will lead to the FFT of your windowed sound having bumps as well, making your FFT harder to interpret.
4. Therfore, we can get better results by multiplying the sound to be analysed with another type of window, whose spectrum is "nicer."

The functions below can be used to generate "nice" windows of any length. Read their help files:


In [ ]:
?np.hamming

In [ ]:
?np.hanning

In [ ]:
?np.blackman

Use these functions to create three window of 1024 samples each. Then plot each window to take a look at the function that is generated. How do they compare?

Create two 1-second sine wave sampled at 100 Hz. The first sine should have a frequency which will exactly match a frequency bin of a 100-point FFT. The second should have a frequency which will *not* exactly match a frequency bin.

(Recall that a 100-point FFT will give you 100 bins. The frequency of the first bin is 0, and the bins evenly divide the frequency space from 0 to the sampling rate into 100 bins.)

In [ ]:
t = # make your time array
s1 = #your first sine wave
s2 = #your second sine wave

Plot the FFT of s1 and s2. How do their shapes compare?

In [ ]:
#Plot your FFTs here

Now, use the following code to apply a 100-point Hamming window to s2, and plot it:

In [ ]:
s2_windowed = np.hamming(100)*s2
plot(s2_windowed)

Now plot the FFT of s2_windowed. How does it compare to the FFT of s2?

In [ ]:
#Plot your FFTs here

Now try computing the FFT of a short snippet of sound from song1 or song2. Compare the FFT with and without windowing. How do they compare?

# Part 3: Using the STFT to implement effects #

a. Use Audacity or anothe program to record yourself saying a few words into an audio file, and save this file as "me.wav", in the same directory as this sketch.

Use the code below to load this file into a Python array and play it:


In [ ]:
me = wavReadMono("me.wav")
play(me)

The code below performs cross synthesis between your voice and song1. Specifically:

* For each 2048-sample analysis frame in song1:
    * Take an FFT analysis frame from song1, and call this f1
    * Take an FFT analysis frame from song2, and call this f2
    * Compute a new spectrum of the same size, by multiplying each (complex) bin of f1 with the magnitude of the same bin of f2
    * Take the IFFT of this spectrum to produce 2048 samples of audio
    * Add this audio to an audio buffer
    * Then move 16 samples forward in time within song1, within your recording, and within the buffer, repeating the process until you reach the end of the song1 file.


In [ ]:
newSound = zeros(size(song1)) #make an audio "buffer" the same length as song1
i = 0 #our index in song1 
j = 0; #our index in "me"
width = 2048; #width of our anlaysis frame
hop = int(2048/16); #hop between subsequent frames
win = np.hamming(2048) #our window :) 

while (i + width < size(song1)) :  #Do this until we get to the end of song1
    if (j + width >= size(me)) : #if we reach the end of "me" first, start at the beginning
        j = 0
    frame1 = win*song1[i:i+width] #select the next 'width' samples from song1 and window them
    frame2 = win*me[j:j+width] #select the next 'width' samples from me and window them
    f1 = fft.fft(frame1) #the next fft of song1 
    f2 = fft.fft(frame2) #the next fft of "me"
    magnitudes = abs(f2) #the magnitudes of "me"
    frame3 = f1 * magnitudes #multiply the complex fft, f1, by the magnitudes of "me", bin by bin
        
    sig1 = fft.ifft(frame3).real #take the inverse FFT to generate audio, and throw away any imaginary garbage, keeping only the real part
    newSound[i:i+width] = newSound[i:i+width] + sig1 #add this to the new sound buffer
    i = int(i + hop) #move to the next analysis location in song1 
    j = int(j + hop) #move to the next analysis location in "me"

newSound = 0.7*newSound/(max(abs(newSound))) #normalise the audio file so it doesn't clip
play(newSound) #play it!
plot(newSound) #plot it!

Make sure you understand the code above. Try changing it! For instance:
* What happens if you use the magnitude of "song1" and the complex FFT of "me" (i.e., switching their roles?)
* Try using two different audio inputs (e.g., two songs, or a voice with white noise, or....)

Think about how you might change this code to make the new sound twice as long, doing time-stretching at the same time.... :)